Block encodings. Qubitization. QSVT.

A Hamiltonian is generally speaking not unitary. A block encoding embets a non-unitary operation $B$ within a larger space as a sub-block (in matrix terms). In doing so we may need to scale $B$ so that it has a magnitude suitable for embedding in a unitary. So suppose that we have done so and embedded $B$ within $U$. Then $U$ acts on the state, assumed to be qubits of which part is an auxillary register in state $\ket{0}$. The result of such an action is a superposition that preserves $\ket{0}$, and an orthogonal state that does not have $\ket{0}$ in the auxillary register. After this application, during measurement, if the auxillary register contains $\ket{0}$, then the main register contains the result of our application of $B$ on them.

If $B$ can be represented as a linear combination of unitaries, then Prep-Select-Prep is a method of block-encoding it

## HS.2.1

In [ ]:
def V_prepare(k):
    """
    Builds the unitary matrix V_k.

    Args:
    - k (float): The value of kappa in the linear combination of two unitaries
    Returns:
    - (numpy.ndarray): The matrix V_k.
    """

    sr_k = np.sqrt(k)
    v_k = np.array([[sr_k, -1], [1, sr_k]]) / np.sqrt(k + 1)

    return v_k


dev = qml.device("default.qubit", wires = 2)

@qml.qnode(dev)
def two_unitary_combo(state, k, U, V):
    """
    Builds the quantum circuit that implements the linear combination of two unitaries.
    Args:
    - state (numpy.ndarray): The state to prepare in the main register
    - k (float): The value of kappa in the linear combination of two unitaries
    - U (numpy.ndarray): The first unitary matrix
    - V (numpy.ndarray): The second unitary matrix
    Returns:
    - (numpy.ndarray): The output state of the circuit
    """

    v_k = V_prepare(k)

    # Apply the unitary v_k
    qml.QubitUnitary(v_k, wires = 0)

    # Prepare the state on the main register (second wire)
    qml.StatePrep(state, wires = 1)

    # Implement the controlled gates
    qml.ControlledQubitUnitary(U, control_wires=0, wires=1, control_values = [0])
    qml.ControlledQubitUnitary(V, control_wires=0, wires=1, control_values = [1])

    # Apply the adjoint of v_k
    qml.adjoint(qml.QubitUnitary(v_k, wires = 0))

    return qml.state()



## HS.2.2a

In [ ]:
def my_select(U_list):
    """
    Implements the SELECT subroutine for the unitaries contained in U_list, from left to right.

    Args:
    - U_list (list(list(complex))): The list of unitary matrices (as a list of lists).
    Returns:
    - No need to return anything.
    """

    ######################
    ### YOUR CODE HERE ###
    ######################
    nc = int(np.ceil(np.log2(len(U_list))))
    nt = int(np.log2(len(U_list[0])))
    for i in range(len(U_list)):
        control_values = [int(x) for x in np.binary_repr(i, width=nc)]
        qml.ControlledQubitUnitary(np.array(U_list[i]), control_wires=range(nc), wires=range(nc, nc + nt), control_values = control_values)


## HS.2.2b

In [ ]:
dev = qml.device("default.qubit")

@qml.qnode(dev)
def my_prepselprep(coeff_list, U_list):
    """
    Implements the PrepSelPrep routine as a PennyLane QNode
    
    Args:
    coeff_list(list(float)): List of positive real numbers, representing the LCU coefficients
    U_list (list(list(complex))): The unitaries in the LCU
    Returns:
    - np.ndarray: The output state of the PrepSelPrep circuit
    """


    ######################
    ### YOUR CODE HERE ###
    ######################
    nc = int(np.ceil(np.log2(len(U_list))))
    pad = 2**nc - len(U_list)
    coeff_list = coeff_list + [0]*pad
    U_list = U_list + [np.eye(len(U_list[0]))]*pad
    l = sum(coeff_list)
    t_state = np.sqrt(np.array(coeff_list) / l)
    
    qml.StatePrep(t_state, wires = range(nc))
    my_select(U_list)
    qml.adjoint(qml.StatePrep(t_state, wires = range(nc)))

    return qml.state()

def apply_lcu(coeff_list, U_list):
    """
    Implements the PrepSelPrep routine as a matrix calculation
    
    Args:
    coeff_list(list(float)): List of positive real numbers, representing the LCU coefficients
    U_list (list(list(complex))): The unitaries in the LCU
    Returns:
    - np.ndarray: The expected output state of PrepSelPrep
    """
    
    lbda = np.sum(coeff_list)
    coeffs_normalized = np.array(coeff_list/lbda)
    lcu = np.sum(np.array([coeffs_normalized[i]*np.array(U_list[i]) for i in range(len(U_list))]), axis = 0)
    k = len(U_list[0])
    state = np.zeros(k)
    state[0] = 1

    return np.dot(lcu, state)

U0 = qml.matrix(qml.IsingXX(0.3,[0,1]))
U1 = qml.matrix(qml.IsingYY(0.4, [0,1]))
coeffs = [1,3,2,4]
ops = [U0, U1, U0, U1]
n = len(U0)

print("for coeff_list = coeffs and U_list = ops, the theoretical output is: ", apply_lcu(coeffs, ops))
print("for coeff_list = coeffs and U_list = ops, the output of your code is: ", my_prepselprep(coeffs, ops)[:n])



## HS.2.3a

In [ ]:
dev = qml.device('default.qubit', wires = 2)

@qml.qnode(dev)
def block_encoding(matrix):
    """
    Encodes a 2 x 2 matrix using a two-qubit circuit (4 x 4 unitary)
    
    Args:
    - matrix (list(list(float))): The 2 x 2 matrix to be encoded
    Returns:
    - np.ndarray: The output state of the encoding circuit.
    """
    
    ######################
    ### YOUR CODE HERE ###
    ######################
    qml.BlockEncode(matrix, wires=range(2))

    return qml.state()

A = [[0.1,0.2],[0.3,0.4]]
circuit_matrix = qml.matrix(block_encoding)(A)
print("The matrix of the circuit is: ", circuit_matrix)



## HS.2.3b

In [ ]:
dev = qml.device("default.qubit")

@qml.qnode(dev)
def prepselprep(coeffs, ops, control):
    """
    Implements the PrepSelPrep routine using PennyLane's built-in functionality
    
    Args:
    - coeff_list(list(float)): List of positive real numbers, representing the LCU coefficients
    - ops (list(qml.operator)): The unitary operators in the LCU (as PennyLane operators)
    - control: The control (auxiliary) wires for the built-in PrepSelPrep routine.
    Returns:
    - np.ndarray: The output state of the PrepSelPrep circuit
    """

    ######################
    ### YOUR CODE HERE ###
    ######################
    qml.PrepSelPrep(qml.ops.LinearCombination(coeffs, ops), control)

    return qml.state()

def apply_lcu(coeff_list, U_list):
    """
    Implements the PrepSelPrep routine as a matrix calculation
    
    Args:
    - coeff_list(list(float)): List of positive real numbers, representing the LCU coefficients
    - U_list (list(list(complex))): The unitaries in the LCU
    Returns:
    - np.ndarray: The expected output state of PrepSelPrep
    """

    lbda = np.sum(coeff_list)
    coeffs_normalized = np.array(coeff_list/lbda)
    lcu = np.sum(np.array([coeffs_normalized[i]*np.array(U_list[i]) for i in range(len(U_list))]), axis = 0)
    k = len(U_list[0])
    state = np.zeros(k)
    state[0] = 1

    return np.dot(lcu, state)

V0 = qml.IsingXX(0.3,[2,3])
V1 = qml.IsingYY(0.4, [2,3])
U0 = qml.matrix(V0)
U1 = qml.matrix(V1)
coeffs = [1,2,2,1]
operators = [V0, V1, V0, V1]
matrices = [U0, U1, U0, U1]
n = len(U0)

print("for coeff_list = coeffs and U_list = ops, the theoretical output is:", apply_lcu(coeffs, matrices))
print("for coeff_list = coeffs and U_list = ops, the output of your code is:", prepselprep(coeffs, operators, [0,1])[:n])

